# Eigenvalues & the Power Method

Let A = $\begin{bmatrix} 6 & 3 & 1 \\
3 & 5 & 2 \\
1 & 2 & 4
\end{bmatrix}$, a symmetric matrix ($A = A^{\top}$) with no entry equal to 0.

In [5]:
import numpy as np

A = np.array([[6, 3, 1], [3, 5, 2], [1, 2, 4]], dtype=float)

w, V = np.linalg.eig(A)
idx = np.argmax(w.real)
lam_star = w[idx].real
v_star = V[:, idx].real
v_star = -v_star if v_star.sum() < 0 else v_star
v_star = v_star / np.linalg.norm(v_star)

print("dominant eigenvalue:", lam_star)
print("dominant eigenvector:", v_star)

dominant eigenvalue: 9.348493934350047
dominant eigenvector: [0.67934373 0.63657455 0.3650547 ]


In [3]:
def power_method(A, epsilon, max_iter=100000, x0=None):
    n = A.shape[0]
    if x0 is None:
        x = np.ones(n)
    else:
        x = np.array(x0, dtype=float)
    
    x = x / np.linalg.norm(x)
    
    prev = x.copy()
    k = 0
    
    while k < max_iter:
        y = A @ x
        x = y / np.linalg.norm(y) 
        rel_change = np.linalg.norm(x - prev) / np.linalg.norm(prev)
        
        if rel_change < epsilon:
            break    
        prev = x.copy()
        k += 1
    
    mu = (x @ (A @ x)) / (x @ x)
    return x, mu, k+1, rel_change


In [8]:
epsilon_values = [1e-3, 1e-6, 1e-9]

for epsilon in epsilon_values:
    v_k, mu_k, iters, rel_change = power_method(A, epsilon)
    
    eig_error = abs(mu_k - lam_star)
    vec_error = np.linalg.norm(v_k - v_star) / np.linalg.norm(v_star)
    
    print("epsilon:", epsilon)
    print("number of iterations:", iters)
    print("eigenvalue error:", eig_error)
    print("eigenvector error:", vec_error)
    print("final relative change:", rel_change)
    
    print("\n")

epsilon: 0.001
number of iterations: 7
eigenvalue error: 7.105817143582271e-07
eigenvector error: 0.0003556325469840886
final relative change: 0.0005356872732264721


epsilon: 1e-06
number of iterations: 14
eigenvalue error: 1.8385293287792592e-12
eigenvector error: 5.726590846818378e-07
final relative change: 8.625343810033565e-07


epsilon: 1e-09
number of iterations: 22
eigenvalue error: 5.329070518200751e-15
eigenvector error: 3.67943953605171e-10
final relative change: 5.541944425457898e-10




# PageRank

In [10]:
A = np.array([
    [0,1,1,1,1],  # page 1
    [0,0,1,1,0],  # page 2
    [1,0,0,1,1],  # page 3
    [1,0,1,0,0],  # page 4
    [1,0,0,1,0]   # page 5
], dtype=float)

row_sums = A.sum(axis=1, keepdims=True)
P = A / row_sums

alpha = 0.85
n = P.shape[0]

# handle dangling rows
P2 = P.copy()
row_sums = P.sum(axis=1, keepdims=True)
dangling = (row_sums.flatten() == 0)

if np.any(dangling):
    P2[dangling, :] = 1.0 / n

# Google matrix
G = alpha * P2 + (1 - alpha) * (np.ones((n, n)) / n)

# power iteration
r = np.ones(n) / n  # uniform start

for _ in range(200):
    r = r @ G
    r = r / r.sum()

print("pagerank:", r.round(4))
print("rank order (1 highest):", (np.argsort(-r)+1))


pagerank: [0.27   0.0874 0.2333 0.2558 0.1535]
rank order (1 highest): [1 4 3 5 2]


In [11]:
print("row sums of P:", P.sum(axis=1))   # should all be 1 (no dangling rows here)
print("row sums of G:", G.sum(axis=1))   # should all be 1
print("sum of r:", r.sum())              # should be 1


row sums of P: [1. 1. 1. 1. 1.]
row sums of G: [1. 1. 1. 1. 1.]
sum of r: 1.0
